# **Molecular Interaction between protein and ligand (Mac)**

## Only use this for a DOWNLOADED or In-house protein-ligand PDB complex, this version is not for DOCKING, and this version do not support one-click ```Run All``` function.

This <font color='orange'>**LABODOCK**</font>**BINDER** notebook is designed for conducting basic molecular docking procedures using **Autodock Vina 1.2.5** and on Mac, please create a separate conda env with the that installed with **vina** by ``` conda install -c conda-forge vina```. Binding interaction analysed with **PLIP 2.3.0**.


## Introduction
labodock_binder is an enhanced iteration of Labodock (https://github.com/RyanZR/labodock), with a primary focus on improved accessibility and user-friendliness. Notable modifications from Labodock version 2.0.0 include:

**Seamless Access**: Unlike the original Google Colab login requirement, labodock_binder allows free access from Chrome, Firefox, and Safari, without the need for signup or login. This is made possible through the implementation of Binder technology (https://mybinder.org/).

**Pre-built Images**: The labodock_binder image is pre-built. When a user clicks the link, the image is pulled from the cloud to the local browser, instead of installing all packages from scratch. This significantly accelerates the readiness for docking.

**Simplified Execution**: labodock_binder streamlines the process with one-click execution, eliminating the need for repetitive variable modifications.

These enhancements collectively contribute to a more user-friendly and accessible experience. labodock_binder emerges as a versatile and convenient tool for users worldwide, requiring only internet availability—no Google account necessary.

**High Safety Standard** :All code executed, data analyzed, and files uploaded during a Binder session, including any modifications to the docking notebook, are automatically deleted when the user logs off or becomes inactive for a few minutes. This ensures that there's no concern about temporary uploads, such as small molecule structures, being leaked. Additionally, Binder retains only the IP addresses associated with the service visits for 30 days, and this information is solely utilized to identify potential abuse of the Binder service. Rest assured that all modifications to the docking notebook are automatically cleared upon closing the browser window, and the platform prioritizes user privacy by minimizing data retention

## Usage

## Please note, this version is ONLY for generating interaction diagram from a given structure not for DOCKING
**PDB_ID** is a four-letter code from the PDB bank website, for example, 3HTB

**Native_lig** below means the native ligand of the co-crystal structure of the PDB bank, it can be found on the PDB website, it should always be a three-letter code, for example, JZ4

**smiles** is ```c1ccccc1``` to serve as a small ligand just for the sake of the completiness of this procedure.

## Please replace the next 3 string value, then from top left **Run** click **Run All Cells**

In [1]:
PDB_ID = "3HTB"

In [2]:
Native_lig = "JZ4"

In [3]:
smiles = "c1ccccc1"

---
---
# **01 | Setting Up Environment**

Firstly, we install all the necessary libraries and packages.

In [4]:
# Internal Modules
import os
import glob
import time
import shutil
# External modules
import py3Dmol
import plip
# Data-related
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
# Docking-related
from rdkit import Chem, RDLogger
from rdkit.Chem import rdFMCS, AllChem, Draw
from spyrmsd import io, rmsd
# Binding interaction-related
from plip.exchange.report import BindingSiteReport
from plip.structure.preparation import PDBComplex
print(f'+ Import completed')

ModuleNotFoundError: No module named 'py3Dmol'

In [ ]:
Job_name = 'Mac_docking_folder4'   # @param {type: 'string'}
invalid_chars = '^<>/\\{}[]~`$ '
assert Job_name, 'Do not leave this blank.'
assert not set(invalid_chars).intersection(Job_name), 'Disallowed characters.'

DIR = os.getcwd()
WRK_DIR = os.path.join(DIR, Job_name)
PRT_FLD = os.path.join(WRK_DIR, 'PROTEIN')
LIG_FLD = os.path.join(WRK_DIR, 'LIGAND')
NTV_FLD = os.path.join(WRK_DIR, 'NATIVE')
DCK_FLD = os.path.join(WRK_DIR, 'DOCKING')
INT_FLD = os.path.join(WRK_DIR, 'INTERACTION')

folders = [WRK_DIR, PRT_FLD, LIG_FLD, NTV_FLD, DCK_FLD, INT_FLD]

for folder in folders:
    if os.path.exists(folder):
        print(f'+ Folder existed: {folder}')
    else:
        os.mkdir(folder)
        print(f'+ Folder created: {folder}')

+ Folder existed: /Users/quantaosun/Desktop/20240125/Mac_docking_folder4
+ Folder existed: /Users/quantaosun/Desktop/20240125/Mac_docking_folder4/PROTEIN
+ Folder existed: /Users/quantaosun/Desktop/20240125/Mac_docking_folder4/LIGAND
+ Folder existed: /Users/quantaosun/Desktop/20240125/Mac_docking_folder4/NATIVE
+ Folder existed: /Users/quantaosun/Desktop/20240125/Mac_docking_folder4/DOCKING
+ Folder existed: /Users/quantaosun/Desktop/20240125/Mac_docking_folder4/INTERACTION


In [ ]:
# @title **Set up utilities**
# @markdown This creates important custom functions and methods for later
# @markdown docking and binding interaction study.

#############################################
# Suppress Warnings

RDLogger.DisableLog('rdApp.warning')

#############################################
# Grid Box Calculation Methods

class GridBox:

    ranges = tuple[list[float], list[float], list[float]]
    coords = tuple[float, float, float]
    center_bxsize = tuple[tuple[float, float, float], tuple[float, float, float]]

    def __init__(self, inpt_file: str) -> None:
        self.inpt = open(inpt_file, 'r')
        self.data = self.inpt.read()
        self.cmol = Chem.MolFromPDBBlock(self.data)
        self.conf = self.cmol.GetConformer()
        self.ntom = self.cmol.GetNumAtoms()
        self.inpt.close()

    def update_gridbox(self, mol_block: str) -> None:
        self.cmol = Chem.MolFromPDBBlock(mol_block)
        self.conf = self.cmol.GetConformer()
        self.ntom = self.cmol.GetNumAtoms()

    def compute_coords(self) -> ranges:
        x_coord = [self.conf.GetAtomPosition(c).x for c in range(self.ntom)]
        y_coord = [self.conf.GetAtomPosition(c).y for c in range(self.ntom)]
        z_coord = [self.conf.GetAtomPosition(c).z for c in range(self.ntom)]
        return x_coord, y_coord, z_coord

    def compute_ranges(self) -> ranges:
        x, y, z = self.compute_coords()
        x_range = [min(x), max(x)]
        y_range = [min(y), max(y)]
        z_range = [min(z), max(z)]
        return x_range, y_range, z_range

    def compute_center(self, use_range: bool = True) -> coords:
        x, y, z = self.compute_ranges() if use_range else self.compute_coords()
        x_center = round(np.mean(x), 3)
        y_center = round(np.mean(y), 3)
        z_center = round(np.mean(z), 3)
        return x_center, y_center, z_center

    def generate_res_molblock(self, residues_list: list[str]) -> str:
        res_lines = [line for line in self.data.split('\n')
                     if line[22:26].lstrip() in residues_list
                     and 'END' not in line]
        res_block = '\n'.join(res_lines)
        return res_block

    def labox(self, scale: float = 2.0) -> coords:
        xr, yr, zr = self.compute_ranges()
        center = self.compute_center()
        bxsize = (round(abs(xr[0] - xr[1]) * scale, 3),
                  round(abs(yr[0] - yr[1]) * scale, 3),
                  round(abs(zr[0] - zr[1]) * scale, 3))
        return center, bxsize

    def eboxsize(self, gy_box_ratio: float = 0.23, modified: bool = False) -> center_bxsize:
        xc, yc, zc = self.compute_coords()
        center = self.compute_center(modified)
        distsq = [(x-center[0])**2 + (y-center[1])**2 + (z-center[2])**2
                  for x, y, z in zip(xc, yc, zc)]
        bxsize = (round(np.sqrt(sum(distsq) / len(xc)) / gy_box_ratio, 3),) * 3
        return center, bxsize

    def autodock_grid(self) -> center_bxsize:
        xr, yr, zr = self.compute_ranges()
        center = self.compute_center()
        bxsize = (22.5, 22.5, 22.5)
        return center, bxsize

    def defined_by_res(self, residue_number: str, scale: float = 1.25) -> center_bxsize:
        res_list = residue_number.replace(',', ' ').split()
        res_block = self.generate_res_molblock(res_list)
        self.update_gridbox(res_block)
        return self.labox(scale=scale)

#############################################
# RMSD Calculation Methods

class ComputeRMSD:

    def __init__(self) -> None:
        self.MCS_mol = None
        self.MCS_png = None

    def load_molecule(self, inpt_file: str, remove_Hs: bool = True) -> tuple:
        molecule = io.loadmol(inpt_file)
        molecule.strip() if remove_Hs else None
        name = os.path.basename(inpt_file).split('.')[0]
        coor = molecule.coordinates
        anum = molecule.atomicnums
        mtrx = molecule.adjacency_matrix
        cmol = Chem.MolFromPDBFile(inpt_file)
        return name, coor, anum, mtrx, cmol

    def mol_to_png(self, mol: object) -> object:
        legend = 'Maximum Common Substructure'
        png = Draw.MolToImage(mol, legend=legend)
        return png

    def find_MCS(self, ref: tuple, lig: tuple) -> object:
        if self.MCS_mol is None:
            MCS_obj = rdFMCS.FindMCS([ref[4], lig[4]])
            MCS_mol = Chem.MolFromSmarts(MCS_obj.smartsString)
            MCS_png = self.mol_to_png(MCS_mol)
            self.MCS_mol = MCS_mol
            self.MCS_png = MCS_png
        return self.MCS_mol

    def hung_RMSD(self, ref: tuple, lig: tuple) -> float:
        try:
            hRMSD = round(rmsd.hrmsd(ref[1], lig[1], ref[2], lig[2]), 3)
        except:
            hRMSD = 'ERROR'
        return hRMSD

    def symm_RMSD(self, ref: tuple, lig: tuple, minimise: bool = False) -> float:
        try:
            sRMSD = round(rmsd.symmrmsd(ref[1], lig[1], ref[2], lig[2], ref[3], lig[3], minimize=minimise), 3)
        except:
            sRMSD = 'ERROR'
        return sRMSD

    def labo_RMSD(self, ref: tuple, lig: tuple) -> float:
        mol_substr = self.find_MCS(ref, lig)
        ref_substr = ref[4].GetSubstructMatch(mol_substr)
        lig_substr = lig[4].GetSubstructMatch(mol_substr)

        distsq = []
        for ref_atom, lig_atom in zip(ref_substr, lig_substr):
            ref_pos = ref[4].GetConformer().GetAtomPosition(ref_atom)
            lig_pos = lig[4].GetConformer().GetAtomPosition(lig_atom)
            ref_coord = np.array((ref_pos.x, ref_pos.y, ref_pos.z))
            lig_coord = np.array((lig_pos.x, lig_pos.y, lig_pos.z))
            coo_dist = np.linalg.norm(ref_coord - lig_coord)
            distsq.append(coo_dist ** 2)

        try:
            lRMSD = round(np.sqrt(sum(distsq)/len(distsq)), 3)
        except:
            lRMSD = 'ERROR'
        return lRMSD

    def rmsd_report(self,
                    ref: tuple,
                    lig: tuple,
                    lRMSD: bool = True,
                    hRMSD: bool = True,
                    sRMSD: bool = True
                    ) -> dict[str: list[float]]:
        report = {}
        report['NAME'] = [lig[0]]
        report['LABO_RMSD'] = [self.labo_RMSD(ref, lig)] if lRMSD else None
        report['HUNG_RMSD'] = [self.hung_RMSD(ref, lig)] if hRMSD else None
        report['SYMM_RMSD'] = [self.symm_RMSD(ref, lig)] if sRMSD else None
        report = {k: v for k, v in report.items() if v is not None}
        return report

#############################################
# AA Consntant and Bond Colour Dictionary

# Kyte and Doolittle Hydropathy Scale (1982)
AA_HB = {'ALA':  1.8, 'ARG': -4.5, 'ASN': -3.5, 'ASP': -3.5, 'CYS':  2.5,
         'GLN': -3.5, 'GLU': -3.5, 'GLY': -0.4, 'HIS': -3.2, 'ILE':  4.5,
         'LEU':  3.8, 'LYS': -3.9, 'MET':  1.9, 'PHE':  2.8, 'PRO': -1.6,
         'SER': -0.8, 'THR': -0.7, 'TRP': -0.9, 'TYR': -1.3, 'VAL':  4.2}

# University of Calgary PI Scale
AA_PI = {'ALA':  6.0, 'ARG': 10.76, 'ASN': 5.41, 'ASP': 2.77, 'CYS': 5.07,
         'GLN': 5.65, 'GLU':  3.22, 'GLY': 5.97, 'HIS': 7.59, 'ILE': 6.02,
         'LEU': 5.98, 'LYS':  9.74, 'MET': 5.74, 'PHE': 5.48, 'PRO':  6.3,
         'SEC': 5.68, 'SER':  5.68, 'THR':  5.6, 'TRP': 5.89, 'TYR': 5.66,
         'VAL': 5.96}

BOND_COL = {'HYDROPHOBIC': ['0x59e382', 'GREEN'],
            'HBOND': ['0x59bee3', 'LIGHT BLUE'],
            'WATERBRIDGE': ['0x4c4cff', 'BLUE'],
            'SALTBRIDGE': ['0xefd033', 'YELLOW'],
            'PISTACKING': ['0xb559e3', 'PURPLE'],
            'PICATION': ['0xe359d8', 'VIOLET'],
            'HALOGEN': ['0x59bee3', 'LIGHT BLUE'],
            'METAL':['0xe35959', 'ORANGE']}

#############################################
# AA-to-Colour Converter Function

def sequential_gradient(value: float,
                        min_value: float,
                        max_value: float,
                        targ_colour: str = '00ff00',
                        interpolation: float = 0.0
                        ) -> str:
    norm_val = (value - min_value) / (max_value - min_value)

    rgb = tuple(int(targ_colour[d:d+2], 16) for d in (0, 2, 4))
    r = int(255 - (255 - rgb[0]) * (1 - interpolation) * norm_val)
    g = int(255 - (255 - rgb[1]) * (1 - interpolation) * norm_val)
    b = int(255 - (255 - rgb[2]) * (1 - interpolation) * norm_val)

    hex_code = f'#{r:02x}{g:02x}{b:02x}'
    return hex_code

def diverging_gradient(value: float,
                       min_value: float,
                       max_value: float,
                       base_colour: str = 'ff0000',
                       targ_colour: str = '0000ff',
                       interpolation: float = 0.3
                       ) -> str:
    norm_val = (value - min_value) / (max_value - min_value)

    white = (255, 255, 255)
    rgb_A = tuple(int(base_colour[d:d+2], 16) for d in (0, 2, 4))
    rgb_B = tuple(int(targ_colour[d:d+2], 16) for d in (0, 2, 4))

    if norm_val < 0.5 - interpolation / 2:
        factor = norm_val / (0.5 - interpolation / 2)
        r = int(rgb_A[0] + (white[0] - rgb_A[0]) * factor)
        g = int(rgb_A[1] + (white[1] - rgb_A[1]) * factor)
        b = int(rgb_A[2] + (white[2] - rgb_A[2]) * factor)
    elif norm_val > 0.5 + interpolation / 2:
        factor = (norm_val - 0.5 - interpolation / 2) / (0.5 - interpolation / 2)
        r = int(white[0] + (rgb_B[0] - white[0]) * factor)
        g = int(white[1] + (rgb_B[1] - white[1]) * factor)
        b = int(white[2] + (rgb_B[2] - white[2]) * factor)
    else:
        r, g, b = white

    hex_code = f'#{r:02x}{g:02x}{b:02x}'
    return hex_code

def a2c_converter(aa_map: dict, grad_func: 'function') -> dict:
    min_value = min(aa_map.values())
    max_value = max(aa_map.values())
    aa_dict = {aa: grad_func(value, min_value, max_value)
               for aa, value in aa_map.items()}
    return aa_dict

#############################################
# Built-in Styling Function

def builtin_style(style: str, opacity: float = 1.0) -> dict:
    match style:
        case _ if any(kw in style for kw in ('Carbon', 'chain', 'ssJmol', 'ssPyMol')):
            style_dict = {'colorscheme': style}
        case 'hydrophobicity':
            style_dict = {'colorscheme': {
                'prop': 'resn', 'map': a2c_converter(AA_HB, sequential_gradient)}}
        case 'isoelectric points':
            style_dict = {'colorscheme': {
                'prop': 'resn', 'map': a2c_converter(AA_PI, diverging_gradient)}}
        case 'b factor':
            style_dict = {'colorscheme': {
                'prop': 'b', 'gradient': 'rwb', 'min': 90, 'max': 50}}
        case _:
            style_dict = {'color': style}

    style_dict.update({'opacity': opacity, 'singleBonds': False})
    return style_dict

#############################################
# Built-in Colour Scale Function

def colour_scale(aa_map: dict, grad_func: 'function') -> None:
    min_value = min(aa_map.values())
    max_value = max(aa_map.values())

    linear_values = np.linspace(min_value, max_value, 100)
    colours = [grad_func(value, min_value, max_value)
               for value in linear_values]

    fig, ax = plt.subplots(figsize=(4.85, 0.25))
    norm_value = plt.Normalize(min_value, max_value)
    colour_map = plt.cm.colors.ListedColormap(colours)
    scalar_map = plt.cm.ScalarMappable(norm_value, colour_map)
    scalar_map.set_array([])

    cscale = plt.colorbar(scalar_map, ax, orientation='horizontal')
    cscale.set_ticks([min_value, max_value])

def show_cscale(rept_info: dict, surf_info: dict) -> None:

    def cs_selector() -> str:
        if any(surf_info):
            style = [*surf_info.values()][0]
        elif any(rept_info):
            style = [*rept_info.values()][0]
        else:
            style = None
        return style

    def cs_display(style: str):
        if style == 'hydrophobicity':
            label_title(style, 'Less', 'More')
            colour_scale(AA_HB, sequential_gradient)
        elif style == 'isoelectric points':
            label_title(style, 'Acid', 'Base')
            colour_scale(AA_PI, diverging_gradient)
        else:
            pass

    def label_title(text: str, min: str, max: str) -> None:
        print(f'-' * 55)
        print(f'{min}{text.upper():^47}{max}')
        print(f'-' * 55)

    cs_display(cs_selector())

#############################################
# Other Functions

def extract_config(inpt_file: str) -> tuple:
    with open(inpt_file, 'r') as inpt:
        data = [line.split() for line in inpt.readlines()]
    center = (float(data[0][2]), float(data[1][2]), float(data[2][2]))
    bxsize = (float(data[4][2]), float(data[5][2]), float(data[6][2]))
    return center, bxsize

def interaction_dict(inpt_file: str, interactions: str = '', usage: str = 'view' or 'lbsp') -> dict:

    usg_map = {'lbsp': 0, 'view': 1}

    def filter_df(int_df: pd.DataFrame, interactions: list = []) -> pd.DataFrame:
        int_df = int_df[int_df['BOND'].isin(interactions)] if interactions else int_df
        return int_df

    def s2f_dict(item: dict) -> dict:
        return {key: tuple(float(val) for val in value[1:-1].split(','))
                for key, value in item.items()}

    def b2c_dict(item: dict) -> dict:
        return {key: BOND_COL[val][usg_map[usage]] for key, val in item.items()}

    intrxn = interactions.replace(',', ' ').split()
    inter_df = pd.read_csv(inpt_file)
    int_dict = filter_df(inter_df, intrxn).to_dict()
    int_dict['LIGCOO'] = s2f_dict(int_dict['LIGCOO'])
    int_dict['PROTCOO'] = s2f_dict(int_dict['PROTCOO'])
    int_dict['COLOR'] = b2c_dict(int_dict['BOND'])

    return int_dict

def find_midpoint(coords: list) -> tuple[float, float, float]:
    return tuple(round(coord, 3) for coord in np.mean(coords, axis=0))

#############################################
# LaboSpace Viewer

class LaboSpace:

    residue_style = {
        'stick':
         {'colorscheme': 'orangeCarbon', 'radius': 0.15}}
    residue_label = {
        'alignment': 'bottomLeft',
        'showBackground': False,
        'inFront': True,
        'fontSize': 14,
        'fontColor': '0x000000',
        'screenOffset': {'x': 25, 'y': 25}}
    atom_label = {
        'alignment': 'bottomLeft',
        'showBackground': False,
        'inFront': True,
        'fontSize': 14,
        'fontColor': '0x000000',
        'screenOffset': {'x': 10, 'y': 10}}

    def __init__(self, vw: int = 500, vh: int = 500) -> None:
        self.mview = py3Dmol.view(width=vw, height=vh)
        self.count = -1
        self.residues = []

    def read_moldata(self, inpt_file: str) -> str:
        inpt = open(inpt_file, 'r')
        data = inpt.read()
        inpt.close()
        return data

    def load_receptor(self, inpt_file: str) -> object:
        data = self.read_moldata(inpt_file)
        self.mview.addModel(data, 'pdb')
        self.count += 1
        return self

    def load_ligand(self, inpt_file: str) -> object:
        data = self.read_moldata(inpt_file)
        self.mview.addModel(data)
        self.count += 1
        return self

    def set_style(self,
                  show_represent: bool = True,
                  represent_type: str = 'cartoon',
                  represent_style: dict = {}
                  ) -> object:
        if show_represent:
            self.mview.setStyle(
                {'model': self.count},
                {represent_type: represent_style})
        else:
            self.mview.setStyle(
                {'model': self.count},
                {})
        return self

    def add_style(self,
                  show_represent: bool = True,
                  represent_style: dict = {}
                  ) -> object:
        if show_represent:
            self.mview.addStyle(
                {'model': self.count},
                represent_style)
        return self

    def add_residues(self,
                     show_residues: bool = True,
                     residue_number: str = ''
                     ) -> object:
        if show_residues and residue_number:
            res = residue_number.replace(',', ' ').split()
            self.residues.extend(list(set(res)))
            self.mview.addStyle(
                {'and': [{'model': self.count}, {'resi': self.residues}]},
                self.residue_style)
            self.mview.addResLabels(
                {'and': [{'model': self.count}, {'resi': self.residues}]},
                self.residue_label)
        return self

    def add_surface(self,
                    show_surface: bool = True,
                    surface_type: str = 'SES',
                    surface_style: dict = {}
                    ) -> object:
        if show_surface:
            self.mview.addSurface(
                surface_type,
                surface_style,
                {'model': self.count})
        return self

    def add_gridbox(self,
                    show_gridbox: bool,
                    center: list[float],
                    bxsize: list[float]
                    ) -> object:
        if show_gridbox:
            bxi, byi, bzi = center
            bxf, byf, bzf = bxsize
            self.mview.addBox({
                'center': {'x': bxi, 'y': byi, 'z': bzi},
                'dimensions': {'w': bxf, 'h': byf, 'd': bzf},
                'color': 'skyBlue',
                'opacity': 0.6})
            self.mview.addLabel(
                f'center: {bxi:>8}, {byi:>8}, {bzi:>8}',
                {'showBackground': False,
                 'fontSize': 14,
                 'fontColor': '0x000000',
                 'useScreen': True,
                 'screenOffset': {'x': 15, 'y': 0}})
            self.mview.addLabel(
                f'bxsize: {bxf:>8}, {byf:>8}, {bzf:>8}',
                {'showBackground': False,
                 'fontSize': 14,
                 'fontColor': '0x000000',
                 'useScreen': True,
                 'screenOffset': {'x': 15, 'y': -20}})
        return self

    def add_interaction(self,
                        interaction_file: str,
                        show_interaction: bool = True,
                        select_interaction: list = []
                        ) -> object:
        if show_interaction:
            int_dict = interaction_dict(interaction_file, select_interaction, 'lbsp')
            dist = int_dict['DIST'].values()
            bond = int_dict['BOND'].values()
            resn = int_dict['RESNR'].values()
            ligcoo = int_dict['LIGCOO'].values()
            prtcoo = int_dict['PROTCOO'].values()
            color = int_dict['COLOR'].values()

            int_res = list(set(resn) - set(self.residues))
            self.residues.extend(int_res)
            self.mview.addStyle(
                {'and': [{'model': 0}, {'resi': int_res}]},
                self.residue_style)
            self.mview.addResLabels(
                {'and': [{'model': 0}, {'resi': int_res}]},
                self.residue_label)

            for dis, col, lig, prt in zip(dist, color, ligcoo, prtcoo):
                mid = find_midpoint([lig, prt])
                self.mview.addCylinder(
                    {'start': {'x': lig[0], 'y': lig[1], 'z': lig[2]},
                     'end': {'x': prt[0], 'y': prt[1], 'z': prt[2]},
                     'radius': 0.05,
                     'fromCap': 1,
                     'toCap': 1,
                     'color': col,
                     'dashed': True})
                self.mview.addLabel(
                    str(dis) + ' Å',
                    {'position': {'x': mid[0], 'y': mid[1], 'z': mid[2]},
                     'alignment': 'bottomLeft',
                     'inFront': False,
                     'backgroundColor': col,
                     'fontSize': 10,
                     'screenOffset': {'x': 10, 'y': 10}})
        return self

    def label_atoms(self, show_label: bool = False) -> object:
        # WARNING: Avoid applying on protein !!!
        if show_label:
            self.mview.addPropertyLabels(
                'atom',
                {'model': self.count},
                self.atom_label)
        return self

    def view_space(self,
                   zoom_model: int = -1,
                   slab_view: bool = False,
                   slab_model: int = -1,
                   background_colour: str = '0xFFFFFF'
                   ) -> None:
        self.mview.setBackgroundColor(background_colour)
        self.mview.setProjection('orthographic')
        self.mview.zoomTo({'model': zoom_model})
        self.mview.fitSlab({'model': slab_model}) if slab_view else None
        self.mview.show()

print(f'+ Methods and functions successfully built')

+ Methods and functions successfully built


---
---
# **02 | Preparing the Receptor**

In this step, we lay the groundwork for molecular docking study by preparing the protein of interest. We will obtain the protein's crystal structure in **`pdb`** format from the [RCSB Protein Data Bank (PDB)](https://www.rcsb.org). Then, we clean the structure by extraction method and seperate its existing subunits into individual files. Finally, we parameterise our target strucutre to generate the **`pdbqt`** file required for docking.

## Prepare the protein (Mac)

In [ ]:
# @title **Download protein PDB file**
# @markdown Enter the **< Accession ID >** to download the protein of interest.

#PDB_ID = '3HTB'  # You can set the default PDB_ID if needed
# PDB_ID = 'your_default_value'  # Uncomment and set your default value

Accession_ID = PDB_ID
PDB_pdb = Accession_ID.upper() + '.pdb'
PDB_pdb_pFile = os.path.join(PRT_FLD, PDB_pdb)
Accession_ID_url = 'http://files.rcsb.org/download/' + PDB_pdb

#!curl {Accession_ID_url} -o {PDB_pdb_pFile} -s

print(f'+ PDB downloaded: {PDB_pdb} > PROTEIN folder')
print(f'+ RCSB PDB link: https://www.rcsb.org/structure/{Accession_ID}')


+ PDB downloaded: 7L10.pdb > PROTEIN folder
+ RCSB PDB link: https://www.rcsb.org/structure/7L10


## Now mannually copy the local protein to the PROTEIN folder

In [ ]:
# @title **Clean protein structure and extract subunits**
# @markdown This generates a **clean** protein structure containing all the subunit(s). \
# @markdown The subunits are then extracted into individual **`pdb`** files. \
# @markdown *Note: `Subunits` and `Chains` are interchangeable to describe the
# @markdown multiple polypeptides that made up the protein.*

def oupt_parse(inpt_file: str) -> tuple:
    name = os.path.basename(inpt_file).split('.')[0]
    path = os.path.dirname(inpt_file)
    dirn = os.path.basename(path)
    return name, dirn, path

def ter_line(asn: str, resn: str, chid: str, resi: str) -> str:
    return f'TER   {asn}      {resn} {chid}{resi:>4} {" "*54}\n'

def extract_entity(inpt_file: str, oupt_file: str, keywords: list) -> None:
    oupt_name, oupt_dirn, oupt_path = oupt_parse(oupt_file)
    extract = []
    headers = ['ATOM', 'HETATM']
    with open(inpt_file, 'r') as inpt:
        for line in inpt:
            record = line.split()[0]
            if all(keyword in line for keyword in keywords) and \
               any(header in record for header in headers):
                extract.append(line)

    assert extract, 'Expected \'keywords\' in PDB lines; found 0 lines'

    with open(oupt_file, 'w') as oupt:
        for line in extract:
            oupt.write(line)
    print(f'+ Entity extracted: {oupt_name}.pdb > {oupt_dirn} folder')

def extract_chains(inpt_file: str) -> None:
    inpt_name, oupt_dirn, oupt_path = oupt_parse(inpt_file)
    with open(inpt_file, 'r') as inpt:
        data = inpt.readlines()

    chid = sorted(set(line[21:22] for line in data))
    chid_leng = len(chid)
    chid_list = ', '.join(chid)
    print(f'+ Chains detected: {chid_leng} ({chid_list})')

    for id in chid:
        oupt_name = inpt_name + '_' + id
        oupt_file = os.path.join(oupt_path, oupt_name + '.pdb')
        with open(oupt_file, 'w') as oupt:
            for line in data:
                if line[21:22] in id:
                    oupt.write(line)
                    asn = f'{int(line[6:11])+1:>5}'
                    resn = line[17:20]
                    resi = line[22:26]
            oupt.write(ter_line(asn, resn, id, resi))
            oupt.write('END')
        print(f'+ Chain extracted: {oupt_name}.pdb > {oupt_dirn} folder')

PDB_prot = Accession_ID + '_prot'
PDB_prot_pdb = PDB_prot + '.pdb'
PDB_prot_pdb_pFile = os.path.join(PRT_FLD, PDB_prot_pdb)

extract_entity(PDB_pdb_pFile, PDB_prot_pdb_pFile, ['ATOM'])
extract_chains(PDB_prot_pdb_pFile)

+ Entity extracted: 7L10_prot.pdb > PROTEIN folder
+ Chains detected: 1 (A)
+ Chain extracted: 7L10_prot_A.pdb > PROTEIN folder


In [ ]:
# @title **Visualise 3D protein structure**
# @markdown Enter the **< Protein >** to be viewed.

#View = '3HTB_prot_A' # @param {type: 'string'}
View = Accession_ID + '_prot_A'
Representation_type = 'cartoon' # @param ['cartoon', 'cross', 'line', 'sphere', 'stick']
Representation_style = 'white' # @param ['chain', 'white', 'whiteCarbon', 'ssJmol', 'ssPyMol', 'b factor', 'hydrophobicity', 'isoelectric points']
Representation_opacity = 1 # @param {type: 'slider', min:0, max:1, step:0.1}
Residue_number = '' # @param {type: 'string'}
Surface_type = 'SES' # @param ['VDW', 'SAS', 'SES', 'MS']
Surface_style = 'hydrophobicity' # @param ['chain', 'white', 'whiteCarbon', 'ssJmol', 'ssPyMol', 'b factor', 'hydrophobicity', 'isoelectric points']
Surface_opacity = 1 # @param {type: 'slider', min:0, max:1, step:0.1}
Show_representation = True # @param {type: 'boolean'}
Show_residue = False # @param {type: 'boolean'}
Show_surface = True # @param {type: 'boolean'}

PROT_view_pFile = os.path.join(PRT_FLD, View + '.pdb')

LBSP = LaboSpace(960, 640)
LBSP.load_receptor(PROT_view_pFile)\
    .set_style(
        show_represent=Show_representation,
        represent_type=Representation_type,
        represent_style=builtin_style(
            style=Representation_style,
            opacity=Representation_opacity))\
    .add_residues(
        show_residues=Show_residue,
        residue_number=Residue_number)\
    .add_surface(
        show_surface=Show_surface,
        surface_type=Surface_type,
        surface_style=builtin_style(
            style=Surface_style,
            opacity=Surface_opacity))
LBSP.view_space()

show_cscale(
    {Show_representation: Representation_style},
    {Show_surface: Surface_style})

In [ ]:
# @title **Parameterise protein with Gasteiger charges**
# @markdown Enter the **< Target Protein >** to be parameterised.\
# @markdown This generate a **`protein.pdbqt`** file after:
# @markdown + Addition of Gasteiger Partial Charge
# @markdown + Addition of polar hydrogens
# @markdown + Removal of non-polar hydrogens

#Target_protein = '3HTB_prot_A' # @param {type: 'string'}
Target_protein = Accession_ID + '_prot_A'
PROT_pdb = Target_protein + '.pdb'
PROT_pdbqt = Target_protein + '.pdbqt'
PROT_pdb_pFile = os.path.join(PRT_FLD, PROT_pdb)
PROT_pdb_dFile = os.path.join(DCK_FLD, PROT_pdb)
PROT_pdbqt_dFile = os.path.join(DCK_FLD, PROT_pdbqt)

!obabel {PROT_pdb_pFile} -xr -O {PROT_pdbqt_dFile} -h --partialcharge gasteiger > /dev/null 2>&1
!mk_prepare_receptor.py --pdbqt {PROT_pdbqt_dFile} -o {PROT_pdbqt_dFile} --skip_gpf > /dev/null 2>&1
print(f'+ Parameterisation: {PROT_pdb} > {PROT_pdbqt}')

shutil.copy(PROT_pdb_pFile, PROT_pdb_dFile)
print(f'+ {PROT_pdbqt} > DOCKING folder')
print(f'+ {PROT_pdb} > DOCKING folder')

---
---
# **03 | Preparing the Native Ligand** (optional)(Mac)

Next, we retrieve the native ligand from the **`pdb`** file to serve as a reference for later comparison. Noted that **this step is optional** as not all **`pdb`** files contain native ligand. However, we highly recommend proceeding with this step for an insightful visual (3D inspection) and qualtitative (RMSD) comparison of binding pose.

In [ ]:
# @title **Retrieve native ligand and extract subunits(code has been modified to hijeck local UNL from local file instead of retreive from online)**
# @markdown Enter the **< Keyword >** assigned for query \
# @markdown This generates a **clean** native ligand structure containing multiple
# @markdown binding poses corresponding to each protein subunit(s). \
# @markdown The binding poses are then extracted and corrected into individual
# @markdown **`pdb`** files.

Keyword = Native_lig # @param {type: 'string'}

def get_molblock(keyword: str) -> str:
    url_path = PRT_FLD
    pdb_file = os.path.join(NTV_FLD, Keyword + '.pdb')
    sdf_file = os.path.join(NTV_FLD, keyword + '.sdf')
    os.system(f'grep {keyword} {PDB_pdb_pFile} -O {pdb_file}')
    os.system(f'obabel -ipdb {pdb_file} -osdf -O {sdf_file}')
    molblock = [mol for mol in  Chem.SDMolSupplier(sdf_file) if mol is not None][0]
    os.remove(sdf_file)
    return molblock

def correct_bond_order(inpt_list: list, temp: Chem.rdchem.Mol) -> None:
    for inpt_file in inpt_list:
        targ = AllChem.MolFromPDBFile(inpt_file)
        cmol = AllChem.AssignBondOrdersFromTemplate(temp, targ)
        pdbb = Chem.MolToPDBBlock(cmol, flavor=4)
        with open(inpt_file, 'w') as oupt:
            oupt.write(pdbb)

true_id = Keyword[-3:] if len(Keyword) > 3 else Keyword
print(f'+ RCSB PDB link: https://www.rcsb.org/ligand/{true_id}')

ntv_pdb = Keyword.upper() + '.pdb'
ntv_pdb_nFile = os.path.join(NTV_FLD, ntv_pdb)
extract_entity(PDB_pdb_pFile, ntv_pdb_nFile, [Keyword, 'HETATM'])
extract_chains(ntv_pdb_nFile)

ntv_nFiles = sorted(glob.glob(NTV_FLD + '/' + Keyword + '_A.pdb'))
ntv_smiles = get_molblock(true_id)
correct_bond_order(ntv_nFiles, ntv_smiles)

In [ ]:
# @title **Visualise 3D native ligand**
# @markdown Enter the **< Native Ligand >** to be viewed.

#View = 'JZ4_A' # @param {type: 'string'}
View = Native_lig + '_A'
Representation_type = 'stick' # @param ['line', 'sphere', 'stick']
Show_atom_labels = False # @param {type: 'boolean'}

NTV_view_nFile = os.path.join(NTV_FLD, View + '.pdb')

LBSP = LaboSpace(960, 640)
LBSP.load_ligand(NTV_view_nFile)\
    .set_style(
        show_represent=True,
        represent_type=Representation_type,
        represent_style={'colorscheme': 'lightGreyCarbon'})\
    .label_atoms(
        show_label=Show_atom_labels)
LBSP.view_space()

## Please go to ```NATIVE``` folder to verify two files have been created

In [ ]:
# @title **Select native ligand**
# @markdown Enter the **< Target Native Ligand >** for later comparison.

#Target_native_ligand = 'JZ4_A' # @param {type: 'string'}
Target_native_ligand = Native_lig + '_A'
NTV_pdb = Target_native_ligand + '.pdb'
NTV_pdb_nFile = os.path.join(NTV_FLD, NTV_pdb)
NTV_pdb_dFile = os.path.join(DCK_FLD, NTV_pdb)

shutil.copy(NTV_pdb_nFile, NTV_pdb_dFile)
print(f'+ {NTV_pdb} > DOCKING folder')

---
---
# **04 | Preparing the Ligand** (Again, be reminded this step is not our focus, just we have to provide something so benzene was chosen since it is small and would fit any possible pocket)

Now, we start to prepare our ligand of interest. We will be using its SMILES notation to generate a 3D energetically minimised ligand **`sd`** file based on specifed force field and steepest gradient alogrithm. We also parameterise our target ligand to generate the **`pdbqt`** file required for docking.

In [ ]:
# @title **Provide ligand information**
# @markdown Enter the **< ID >** and its **< SMILES >** notation.

ID = 'small' # @param {type: 'string'}
SMILES = smiles # @param {type: 'string'}

assert ID.strip(), 'Expected \'ID\'; found \'none\''
assert SMILES.strip(), 'Expected \'SMILES\'; found \'none\''

print(f'+ Ligand \'{ID}\' with \'{SMILES}\' ready for SDF generation')

+ Ligand 'small' with 'c1ccccc1' ready for SDF generation


In [ ]:
# @title **Generate ligand SD file**
# @markdown Select **< Force Field >** to set MM method for energy minimisation. \
# @markdown Select **< Convergence Criteria >** to set minimum iteration difference required before stopping. **[ Default is 0.00001 ]** \
# @markdown Select **< Maximum Steps >** to set maximum iterations allowed for minimisation. **[ Default is 10000 ]**

Force_field = 'UFF' # @param ['GAFF', 'Ghemical', 'MMFF94', 'MMFF94s', 'UFF']
Convergence_criteria = '0.00001' # @param ['0.1', '0.01','0.001', '0.0001', '0.00001', '0.000001', '0.0000001']
Maximum_steps = 10000 # @param {type: 'slider', min:1000, max:100000, step:1000}

def check_convergence(log_file: str) -> None:
    with open(log_file, 'r') as inpt:
        data = inpt.read()
    if 'CONVERGED' in data:
        verb = 'has'
        step = data.split('\n')[-4].split()[0]
        step_str = f'({step} steps)'
    else:
        verb = 'has not'
        step_str = ''
    print(f'+ Steepest gradient {verb} converged {step_str}')

LIG_ID = ID
LIG_sdf = LIG_ID + '.sdf'
LIG_obmin_log = LIG_ID + '_obmin.log'
LIG_sdf_lFile = os.path.join(LIG_FLD, LIG_sdf)
LIG_obmin_log_lFile = os.path.join(LIG_FLD, LIG_obmin_log)

print(f'+ Selected {Force_field} for energy minimisation '
      f'up to {Convergence_criteria} iteration difference or '
      f'at most {Maximum_steps:,} steps')

!obabel -:{'"'+SMILES+'"'} -O {LIG_sdf_lFile} --title {LIG_ID} --gen3d \
--best --minimize --ff {Force_field} --steps {Maximum_steps} --sd \
--crit {Convergence_criteria} --log &> {LIG_obmin_log_lFile}

check_convergence(LIG_obmin_log_lFile)
print(f'+ {LIG_sdf} > LIGAND folder')

+ Selected UFF for energy minimisation up to 0.00001 iteration difference or at most 10,000 steps
+ Steepest gradient has converged (20 steps)
+ small.sdf > LIGAND folder


In [ ]:
# @title **Visualise 3D ligand**
# @markdown Enter the **< Ligand >** to be viewed.

View = 'small' # @param {type: 'string'}
Representation_type = 'stick' # @param ['line', 'sphere', 'stick']
Show_atom_labels = False # @param {type: 'boolean'}

LIG_view_lFile = os.path.join(LIG_FLD, View + '.sdf')

LBSP = LaboSpace(960, 640)
LBSP.load_ligand(LIG_view_lFile)\
    .set_style(
        show_represent=True,
        represent_type=Representation_type,
        represent_style={'colorScheme': 'lightGreyCarbon'})\
    .label_atoms(
        show_label=Show_atom_labels)
LBSP.view_space()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
# @title **Parameterise ligand with Gasteiger charges**
# @markdown This generates a **`ligand.pdbqt`** file after:
# @markdown + Addition of Gasteiger Partial Charge
# @markdown + Removal of non-polar hydrogens

def pdbqt_add_chid(inpt_file: str) -> None:
    with open(inpt_file, 'r') as inpt:
        data = inpt.read()
    new_data = data.replace('  UNL  ', '  UNL A')
    with open(inpt_file, 'w') as oupt:
        oupt.write(new_data)

LIG_pdbqt = ID + '.pdbqt'
LIG_dFLD = os.path.join(DCK_FLD, ID)
LIG_pdbqt_dFFile = os.path.join(LIG_dFLD, LIG_pdbqt)

os.makedirs(LIG_dFLD, exist_ok=True)
!mk_prepare_ligand.py -i {LIG_sdf_lFile} -o {LIG_pdbqt_dFFile} > /dev/null 2>&1
pdbqt_add_chid(LIG_pdbqt_dFFile)

print(f'+ Parameterisation: {LIG_sdf} > {LIG_pdbqt}')
print(f'+ {LIG_pdbqt} > DOCKING folder')

+ Parameterisation: small.sdf > small.pdbqt
+ small.pdbqt > DOCKING folder


---
---
# **05 | Setting Up Grid Box** (Similarly this is not our focus here but just for the sake of taking advantage of this workflow we have to run this)

Here, we define a chemical search space with the use of grid box, which often centered within the binding, active or allosteric site of the target protein.

Built-in methods for defining grid box include:

+ **`LaBOX`** : Use the mean of XYZ extreme values from native ligand. [Link](https://github.com/RyanZR/LaBOX)
+ **`eBoxSize`** : Use the radius of gyration of native ligand. [Link](https://github.com/michal-brylinski/eboxsize)
+ **`eBoxSize-Mod`** : Similar to eBoxSize, but box center is computed using LaBOX method.
+ **`Autodock-Grid`** : 22.5 × 22.5 × 22.5 Å
+ **`Defined-by-Res`** : Use when important residues involved in binding interaction are known.
+ **`Manual-Mode`** : Use the sliders below to define grid box.

In [ ]:
# @title **Place grid box at binding site**
# @markdown Select **< Method >** to set grid box at the center of binding site.

Residue_number = '' # @param {type: 'string'}
Method = 'LaBOX' # @param ['LaBOX', 'eBoxSize', 'eBoxSize-Mod', 'Autodock-Grid', 'Defined-by-Res', 'Manual-Mode']
Show_residues = True # @param {type: 'boolean'}

# @markdown **Manual Mode**
X = 0 # @param {type: 'number'}
Y = 0 # @param {type: 'number'}
Z = 0 # @param {type: 'number'}
Width = 10 # @param {type: 'number'}
Height = 10 # @param {type: 'number'}
Depth = 10 # @param {type: 'number'}
Scale = 1.0 # @param {type: 'number'}

LBSP = LaboSpace(960, 640)
LBSP_view = True

try:
    if Method == 'Defined-by-Res':
        GB = GridBox(PROT_pdb_dFile)
        center, bxsize = GB.defined_by_res(Residue_number)
    if Method == 'Manual-Mode':
        center = (X, Y, Z)
        bxsize = (round(Width, 3),
                  round(Height, 3),
                  round(Depth, 3))
except Exception as excp:
    print(f'+ {excp}')
    print(f'+ Error can be:\n'
           '  - Expected \'Residue_number\'; found \'none\'\n'
           '  - Expected \'protein PDB file\'; found \'none\'\n'
           '  - Due to atomic error in \'protein PDB file\'')
    LBSP_view = False
else:
    LBSP.load_receptor(PROT_pdb_dFile)\
        .set_style(
            show_represent=True,
            represent_type='cartoon',
            represent_style={'color': 'white'})\
        .add_residues(
            show_residues=Show_residues,
            residue_number=Residue_number)

try:
    if Method == 'LaBOX':
        GB = GridBox(NTV_pdb_nFile)
        center, bxsize = GB.labox()
    if Method == 'eBoxSize':
        GB = GridBox(NTV_pdb_nFile)
        center, bxsize = GB.eboxsize()
    if Method == 'eBoxSize-Mod':
        GB = GridBox(NTV_pdb_nFile)
        center, bxsize = GB.eboxsize(modified=True)
    if Method == 'Autodock-Grid':
        GB = GridBox(NTV_pdb_nFile)
        center, bxsize = GB.autodock_grid()
except Exception as excp:
    print(f'+ {excp}')
    print(f'+ Expected \'native ligand PDB file\'; found \'none\'')
    print(f'+ Upload file to \'{NTV_FLD}\' and run \'Select native ligand\' cell to load it.')
    LBSP_view = False
else:
    try:
        LBSP.load_ligand(NTV_pdb_nFile)\
            .set_style(
                show_represent=True,
                represent_type='stick',
                represent_style={'colorScheme': 'greyCarbon'})
    except:
        pass

if LBSP_view:
    LBSP.add_gridbox(show_gridbox=True, center=center, bxsize=bxsize)
    LBSP.view_space()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
# @title **Generate docking config file**
# @markdown This generates **`config_file`** for AutoDock Vina.

cfg_name = 'config_file'
cfg_dFile = os.path.join(DCK_FLD, cfg_name)

with open(cfg_dFile, 'w') as cfg:
    cfg.write(f'center_x = {center[0]}\n')
    cfg.write(f'center_y = {center[1]}\n')
    cfg.write(f'center_z = {center[2]}\n')
    cfg.write(f'\n')
    cfg.write(f'size_x = {bxsize[0]}\n')
    cfg.write(f'size_y = {bxsize[1]}\n')
    cfg.write(f'size_z = {bxsize[2]}\n')

print(f'+ {cfg_name} > DOCKING folder')

+ config_file > DOCKING folder


---
---
# **06 | Performing Molecular Docking** (Right, this again not our focus but we just run it so all the variables are there)

We now proceed with performing molecular docking using Autodock Vina for our target ligand. The duration can vary, mainly influenced by factors including **number of rotatable bonds** and **level of exhaustiveness** selected. Do be patient as the process may take a few minutes to complete.

- Note, on Mac, we assume you already installed vina from conda-forge

In [ ]:
# @title **Run AutoDock Vina**
# @markdown Select **< Exhaustiveness >** for molecular docking. **[ Default is 16 ]** \
# @markdown This generates a **`ligand_output.pdbqt`** containing different binding poses predicted.

Exhaustiveness = '16' # @param [4, 8, 16, 32, 64, 128, 256]

cpu_cores = os.cpu_count()
LIG_oupt_log = LIG_ID + '_output.log'
LIG_oupt_pdbqt = LIG_ID + '_output.pdbqt'
LIG_oupt_log_dFFile = os.path.join(LIG_dFLD, LIG_oupt_log)
LIG_oupt_pdbqt_dFFile = os.path.join(LIG_dFLD, LIG_oupt_pdbqt)

# -- Start docking --
start = time.time()
!vina --receptor {PROT_pdbqt_dFile} --ligand {LIG_pdbqt_dFFile} \
--out {LIG_oupt_pdbqt_dFFile} --config {cfg_dFile} --cpu {cpu_cores} \
--exhaustiveness {Exhaustiveness} --verbosity 2 | tee {LIG_oupt_log_dFFile}
end = time.time()
# -- End docking --

print(f'')
print(f'+ Docking completed')
print(f'+ Time elapsed: ' + time.strftime('%Mm %Ss', time.gmtime(end - start)))
print(f'+ {LIG_oupt_pdbqt} > DOCKING folder')

AutoDock Vina v1.2.5
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp. Chem. (2010)                         #
# DOI 10.1002/jcc.21334                                         #
#                                                               #
# Please see https://github.com/ccsb-scripps/AutoDock-V

In [ ]:
# @title **Process output file**
# @markdown This generates **`ligand_[n].pdb`** for each predicted binding poses.

LIG_dash_pdb = LIG_ID + '_.pdb'
LIG_dash_pdb_dFFile = os.path.join(LIG_dFLD, LIG_dash_pdb)

with open(LIG_oupt_pdbqt_dFFile, 'r') as oupt:
    data = oupt.read()
    pose = data.count('MODEL')

!obabel {LIG_oupt_pdbqt_dFFile} -O {LIG_dash_pdb_dFFile} -m > /dev/null 2>&1
print(f'+ {pose} {LIG_ID}_[n].pdb > DOCKING folder')

+ 9 small_[n].pdb > DOCKING folder


## Go to ```DOCKING``` folder (In terminal)

```
cd ./small

 !cp ../XEY_A.pdb .

 #  Just copy the native ligand to the docked folder, the name may varies

 !babel -ipdb XEY_A.pdb -opdb -O XEY_A_obabel.pdb


 ```

 ## Now open the XEY_A_obabel.pdb and replace all UNK to UNL with sublime, also add chain number 1 as in other small_1.pdb, as well as the A column

The small_0.pdb after sublime modification

```
HETATM    1  C4  UNL A   1      20.579  42.813  10.767  1.00  0.00           C  
HETATM    2  C5  UNL A   1      20.925  42.053   9.652  1.00  0.00           C  
HETATM    3  C6  UNL A   1      20.554  42.499   8.392  1.00  0.00           C  
HETATM    4  N1  UNL A   1      21.735  41.218  13.882  1.00  0.00           N  
HETATM    5  C7  UNL A   1      19.864  43.693   8.289  1.00  0.00           C  
HETATM    6  C8  UNL A   1      19.548  44.385   9.461  1.00  0.00           C  
HETATM    7  N2  UNL A   1      16.685  47.856  11.029  1.00  0.00           N  
HETATM    8  C9  UNL A   1      18.740  45.506   9.092  1.00  0.00           C  
HETATM    9  C10 UNL A   1      18.686  45.430   7.740  1.00  0.00           C  
HETATM   10  C11 UNL A   1      18.161  46.467  10.022  1.00  0.00           C  
HETATM   11  C12 UNL A   1      16.969  47.216   9.913  1.00  0.00           C  
HETATM   12  N3  UNL A   1      17.705  47.541  11.867  1.00  0.00           N  
HETATM   13  C13 UNL A   1      17.761  48.107  13.212  1.00  0.00           C  
HETATM   14  C14 UNL A   1      18.002  47.071  14.287  1.00  0.00           C  
HETATM   15  C15 UNL A   1      17.334  45.854  14.281  1.00  0.00           C  
HETATM   16  N4  UNL A   1      19.901  43.971  10.681  1.00  0.00           N  
HETATM   17  C   UNL A   1      20.708  42.442  15.717  1.00  0.00           C  
HETATM   18  N   UNL A   1      20.949  42.241  14.300  1.00  0.00           N  
HETATM   19  C3  UNL A   1      21.704  41.311  12.572  1.00  0.00           C  
HETATM   20  C2  UNL A   1      20.917  42.393  12.134  1.00  0.00           C  
HETATM   21  C1  UNL A   1      20.458  42.963  13.286  1.00  0.00           C  
HETATM   22  O   UNL A   1      19.361  44.362   7.211  1.00  0.00           O  
HETATM   23  C20 UNL A   1      18.591  46.708  11.309  1.00  0.00           C  
HETATM   24  C19 UNL A   1      18.835  47.353  15.355  1.00  0.00           C  
HETATM   25  C18 UNL A   1      18.947  46.468  16.422  1.00  0.00           C  
HETATM   26  C17 UNL A   1      18.260  45.281  16.409  1.00  0.00           C  
HETATM   27  C16 UNL A   1      17.481  44.960  15.324  1.00  0.00           C  
CONECT    1    2    2   16   20                                       
CONECT    2    1    1    3                                            
CONECT    3    2    5    5                                            
CONECT    4   18   19   19                                            
CONECT    5    3    3    6   22                                       
CONECT    6    5    8   16   16                                       
CONECT    7   11   11   12                                            
CONECT    8    6    9    9   10                                       
CONECT    9    8    8   22                                            
CONECT   10    8   11   23   23                                       
CONECT   11    7    7   10                                            
CONECT   12    7   13   23                                            
CONECT   13   12   14                                                 
CONECT   14   13   15   15   24                                       
CONECT   15   14   14   27                                            
CONECT   16    1    6    6                                            
CONECT   17   18                                                      
CONECT   18    4   17   21                                            
CONECT   19    4    4   20                                            
CONECT   20    1   19   21   21                                       
CONECT   21   18   20   20                                            
CONECT   22    5    9                                                 
CONECT   23   10   10   12                                            
CONECT   24   14   25   25                                            
CONECT   25   24   24   26                                            
CONECT   26   25   27   27                                            
CONECT   27   15   26   26                                            
MASTER        0    0    0    0    0    0    0    0   27    0   27    0
```

## Copy this block to the clipbord and we will used it in the next step

---
---
# **07 | Profiling Binding Interactions**

After docking, we use PLIP to determine the non-covalent binding interactions between the docked ligand and target protein. We will prepare the required complex **`pdb`** file containing docked ligand pose and target protein for PLIP to profile interactions.

In [ ]:
# @title **Generate complex PDB file**
# @markdown This merges the target protein with each docked ligand poses to
# @markdown produce individual **`cmpx.pdb`** file.

def generate_cmpx_pdb(inpt_prot: str, inpt_pose: str, oupt_cmpx: str) -> None:

    def write_line(line: str, keywords: list, oupt_file: str) -> None:
        header = line.split()[0]
        if header in keywords:
            oupt_file.write(line)

    def cmpx_writer() -> None:
        with open(oupt_cmpx, 'w') as oupt_file, \
             open(inpt_prot, 'r') as prot_file, \
             open(inpt_pose, 'r') as pose_file:
            for prot_line in prot_file:
                write_line(prot_line, prot_headers, oupt_file)
            for pose_line in pose_file:
                write_line(pose_line, pose_headers, oupt_file)

    prot_headers = ['ATOM', 'CONECT', 'TER']
    pose_headers = ['ATOM', 'CONECT', 'END']
    cmpx_writer()

LIG_iFLD = os.path.join(INT_FLD, LIG_ID)
os.makedirs(LIG_iFLD, exist_ok=True)
LIG_pdb_dFFiles = sorted(glob.glob(LIG_dFLD + '/*.pdb'))

count = 0
for pose_file in tqdm(LIG_pdb_dFFiles):
    pose_name = os.path.basename(pose_file).split('.')[0]
    cmpx_pdb = pose_name + '_cmpx.pdb'
    cmpx_pdb_iFile = os.path.join(LIG_iFLD, cmpx_pdb)
    generate_cmpx_pdb(PROT_pdb_dFile, pose_file, cmpx_pdb_iFile)
    count += 1

print(f'+ {count} {LIG_ID}_[n]_cmpx.pdb > INTERACTION folder')

  0%|          | 0/11 [00:00<?, ?it/s]

+ 11 small_[n]_cmpx.pdb > INTERACTION folder


## It says 11 structures were successfully generated, but actually, the one with the native ligand (small_1_cmpx.pdb, do not contain any ligand if you open with sublime and check) failed.

## Now go to the ```INTERACTION``` folder

### Modify small_1_cmpx.pdb by deleting all CONNECTS from the end, and paste what you have copied from the native ligadn. Delete the ```small_1_cmpx.pdb``` and rename ```XEY_A_obabel_cmpx.pdb``` to ``small_1_cmpx.pdb```

### Delete any other files, there should only 9 files

run the next block to generate the interaction csv file from PLIP

In [ ]:
# @title **Run PLIP**
# @markdown This generates an **`interpt.csv`** containing all detected binding
# @markdown interaction for each individual **`cmpx.pdb`** file. \
# @markdown Supported: **`HYDROPHOBIC`**, **`HBOND`**, **`WATERBRIDGE`**,
# @markdown **`SALTBRIDGE`**, **`PISTACKING`**, **`PICATION`**, **`HALOGEN`**, **`METAL`**. \

def interaction_profiler(inpt_cmpx: str, oupt_csv: str) -> None:

    int_bonds = ['HYDROPHOBIC', 'HBOND', 'WATERBRIDGE', 'SALTBRIDGE',
                 'PISTACKING', 'PICATION', 'HALOGEN', 'METAL']

    def BSR(inpt_cmpx: str) -> object:
        cmpx_mol = PDBComplex()
        cmpx_mol.load_pdb(inpt_cmpx)
        cmpx_lig = [lig for lig in cmpx_mol.ligands if lig.hetid == 'UNL'][0]
        cmpx_mol.characterize_complex(cmpx_lig)
        cmpx_int = cmpx_mol.interaction_sets['UNL:A:1']
        cmpx_rpt = BindingSiteReport(cmpx_int)
        return cmpx_rpt

    def BSR_feat(bsr: object) -> list:
        return [list(getattr(bsr, bond.lower() + '_features')) + ['BOND']
                for bond in int_bonds]

    def BSR_info(bsr: object) -> list:
        return [[list(info) + [bond] for info in getattr(bsr, bond.lower() + '_info')]
                for bond in int_bonds]

    def replace_column(df: pd.DataFrame, col_A: str, cols: list) -> pd.DataFrame:
        for col in cols:
            if col in df.columns:
                df[col_A] = df[col_A].fillna(df[col])
            else:
                pass
        return df

    def BSR_dataframe(bsr_feat: list, bsr_info: list) -> pd.DataFrame:
        bsr_df = []
        for feat, info in zip(bsr_feat, bsr_info):
            if info:
                df = pd.DataFrame(info, columns=feat)
            else:
                df = pd.DataFrame(columns=bsr_feat[0])
            bsr_df.append(df)
        BSR_df = pd.concat(bsr_df, ignore_index=True)
        BSR_df = replace_column(BSR_df, 'DIST', ['DIST_D-A', 'CENTDIST'])
        return BSR_df

    cmpx_bsr = BSR(inpt_cmpx)
    bsr_feat = BSR_feat(cmpx_bsr)
    bsr_info = BSR_info(cmpx_bsr)
    bsr_data = BSR_dataframe(bsr_feat, bsr_info)
    bsr_data.to_csv(oupt_csv, index=False)

cmpx_pdb_iFFiles = sorted(glob.glob(LIG_iFLD + '/*_cmpx.pdb'))

count = 0
for cmpx_file in tqdm(cmpx_pdb_iFFiles):
    cmpx_name = os.path.basename(cmpx_file).split('.')[0][:-5]
    inter_csv = cmpx_name + '_interpt.csv'
    inter_csv_iFile = os.path.join(LIG_iFLD, inter_csv)
    interaction_profiler(cmpx_file, inter_csv_iFile)
    count += 1

print(f'+ {count} {LIG_ID}_[n]_interpt.csv > INTERACTION folder')

  0%|          | 0/9 [00:00<?, ?it/s]

/var/folders/6l/wn29njjn52s5bm1x_qwy9qk00000gn/T/ipykernel_92587/2616583659.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  BSR_df = pd.concat(bsr_df, ignore_index=True)


+ 9 small_[n]_interpt.csv > INTERACTION folder


---
---
# **08 | Analysing Docking Results**

We process all the docking and binding internaction results generated into for further evaluation. We will compute the RMSD between native ligand and docked ligands using three different calculation methods. We will also visualise the docking pose in three-dimensional chemical space.

In [ ]:
# @title **Show interaction profile**
# @markdown Enter the **< Pose Name >** to view the binding interactions.

#Pose_name = 'JZ4' # @param {type : 'string'}
Ligand_name = ID
Pose_name = Ligand_name + '_1'
LIG_interpt_csv = Pose_name + '_interpt.csv'
LIG_interpt_csv_iFile = os.path.join(LIG_iFLD, LIG_interpt_csv)
LIG_interpt_dict = interaction_dict(LIG_interpt_csv_iFile, usage='view')
LIG_interpt_df = pd.DataFrame.from_dict(LIG_interpt_dict)
LIG_interpt_df = LIG_interpt_df[['RESTYPE', 'RESNR', 'DIST', 'BOND', 'COLOR']]
LIG_interpt_df

,RESTYPE,RESNR,DIST,BOND,COLOR
0,THR,25,3.45,HYDROPHOBIC,GREEN
1,GLY,143,3.15,HBOND,LIGHT BLUE
2,CYS,145,3.14,HBOND,LIGHT BLUE
3,GLU,166,2.84,HBOND,LIGHT BLUE
4,HIS,41,5.21,PISTACKING,PURPLE


---
---
# **09 | Visualising Docking Results**

We process all the docking and binding internaction results generated into for further evaluation. We will compute the RMSD between native ligand and docked ligands using three different calculation methods. We will also visualise the docking pose in three-dimensional chemical space.

In [ ]:
# @title **Visualise 3D docking pose**
# @markdown **PROTEIN MODEL** \
# @markdown Enter the **< Protein >** to be viewed.

#View_protein = '3HTB_prot_A' # @param {type: 'string'}
View_protein = PDB_ID + '_prot_A'
Protein_type = 'cartoon' # @param ['cartoon', 'cross', 'line', 'sphere', 'stick']
Protein_style = 'white' # @param ['chain', 'white', 'whiteCarbon', 'ssJmol', 'ssPyMol', 'b factor', 'hydrophobicity', 'isoelectric points']
Protein_opacity = 0.6 # @param {type: 'slider', min:0, max:1, step:0.1}
Residue_number = '' # @param {type: 'string'}
Surface_type = 'SES' # @param ['VDW', 'SAS', 'SES', 'MS']
Surface_style = 'isoelectric points' # @param ['chain', 'white', 'whiteCarbon', 'ssJmol', 'ssPyMol', 'b factor', 'hydrophobicity', 'isoelectric points']
Surface_opacity = 0.95 # @param {type: 'slider', min:0, max:1, step:0.1}
Show_protein = True # @param {type: 'boolean'}
Show_residue = False # @param {type: 'boolean'}
Show_surface = True # @param {type: 'boolean'}
Show_gridbox = False # @param {type: 'boolean'}

# @markdown **LIGAND MODEL** \
# @markdown Enter the **< Ligand >** to be viewed.

#View_native_ligand = 'JZ4_A' # @param {type: 'string'}
View_native_ligand = Native_lig + '_A'
Native_ligand_style = 'stick' # @param ['cross', 'line', 'sphere', 'stick']
Show_native_ligand = True # @param {type: 'boolean'}
#View_docked_ligand = 'small_1' # @param {type: 'string'}
View_docked_ligand = ID + '_1'
Docked_ligand_style = 'stick' # @param ['cross', 'line', 'sphere', 'stick']
Show_docked_ligand = False # @param {type: 'boolean'}

# @markdown **INTERACTION MODEL** \
# @markdown Enter the **< Interaction Type >** to be viewed. \
# @markdown Select or combine from **`HYDROPHOBIC`**, **`HBOND`**, **`WATERBRIDGE`**,
# @markdown **`SALTBRIDGE`**, **`PISTACKING`**, **`PICATION`**, **`HALOGEN`**, **`METAL`**. \
# @markdown *Note: All interactions are selected if not provided.*

Interaction_type = "HBOND, WATERBRIDGE, SALTBRIDGE, PISTACKING, PICATION, HALOGEN, METAL" # @param {type: 'string'}
Show_interaction = True # @param {type: 'boolean'}

# @markdown **OTHER OPTIONS** \
# @markdown Miscellaneous visualisation settings.

Slab_view = False # @param {type: 'boolean'}

dLIG_dFLD = os.path.join(DCK_FLD, View_docked_ligand[:-2])
iLIG_iFLD = os.path.join(INT_FLD, View_docked_ligand[:-2])
PROT_view_dFile = os.path.join(DCK_FLD, View_protein + '.pdb')
NTV_view_dFile = os.path.join(DCK_FLD, View_native_ligand + '.pdb')
LIG_view_dFFile = os.path.join(dLIG_dFLD, View_docked_ligand + '.pdb')
INT_view_iFFile = os.path.join(iLIG_iFLD, View_docked_ligand + '_interpt.csv')
cfg_center, cfg_bxsize = extract_config(cfg_dFile)

LBSP = LaboSpace(960, 640)
LBSP.load_receptor(PROT_view_dFile)\
    .set_style(
        show_represent=Show_protein,
        represent_type=Protein_type,
        represent_style=builtin_style(
            style=Protein_style,
            opacity=Protein_opacity))\
    .add_residues(
        show_residues=Show_residue,
        residue_number=Residue_number)\
    .add_surface(
        show_surface=Show_surface,
        surface_type=Surface_type,
        surface_style=builtin_style(
            style=Surface_style,
            opacity=Surface_opacity))

try:
    LBSP.load_ligand(NTV_view_dFile)\
        .set_style(
            show_represent=Show_native_ligand,
            represent_type=Native_ligand_style,
            represent_style={'colorscheme': ''})
except:
    pass

LBSP.load_ligand(LIG_view_dFFile)\
    .set_style(
        show_represent=Show_docked_ligand,
        represent_type=Docked_ligand_style,
        represent_style={'colorscheme': 'salmonCarbon'})
LBSP.add_interaction(
    interaction_file=INT_view_iFFile,
    show_interaction=Show_interaction,
    select_interaction=Interaction_type)
LBSP.add_gridbox(
    show_gridbox=Show_gridbox,
    center=cfg_center,
    bxsize=cfg_bxsize)
LBSP.view_space(
    slab_view=Slab_view)

show_cscale({Show_protein: Protein_style}, {Show_surface: Surface_style})

# References:

- AutoDock Vina 1.2.5 (https://vina.scripps.edu/)
- Meeko 0.5.0 (https://github.com/forlilab/Meeko)
- OpenBabel 3.1.1 (https://github.com/openbabel/openbabel)
- PLIP 2.3.0 (https://plip-tool.biotec.tu-dresden.de/plip-web/plip/index)
- Py3Dmol 2.0.3 (https://pypi.org/project/py3Dmol/)
- Rdkit 2023.9.1 (https://github.com/rdkit/rdkit)
- Spyrmsd 0.6.0 (https://github.com/RMeli/spyrmsd)

- This browser docking experience wouldn't be possible without using Binder (https://mybinder.org/)